In [1]:
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque
import sqlite3
import pandas as pd
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
nltk.download('stopwords')
import re
import matplotlib.pyplot as plt
from collections import Counter

conn = sqlite3.connect('twitter.db',check_same_thread=False)
c = conn.cursor()

#popular topics: google, olympics, trump, gun, usa

app = dash.Dash(__name__)

app.layout = html.Div(
    [   html.H2('Live Twitter Sentiment'),
        dcc.Input(id='sentiment_term', value='usa', type='text'),
     
        dcc.Graph(id='live-graph',animate=True),
        dcc.Interval(
            id='graph-update',
            interval=1*1000,
            n_intervals= 0
        ),
     
        html.H2('Trending Words'),
        dcc.Graph(id='discussion',animate=True),
        dcc.Interval(id='topic_update',
                    interval = 10*1000,
                    n_intervals = 0),
     
         html.H2('Pie-Chart Sentiment Tracker'),
         dcc.Graph(id='sentiment_pie',animate=False),
         dcc.Interval(id='pie_sentiment',
                     interval = 3*1000,
                     n_intervals = 0)
    ]
)

@app.callback(Output('live-graph', 'figure'),
              [Input('sentiment_term','value'),
              Input('graph-update', 'n_intervals')
              ])

def update_graph_scatter(sentiment_term,n_intervals):
    try:
        
        
        df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
        df.sort_values('unix', inplace=True)
        df['sentiment_smoothed'] = df['sentiment'].rolling(int(len(df)/5)).mean()
        df.dropna(inplace=True)
   
        X = df.unix
        Y = df.sentiment_smoothed

        data = plotly.graph_objs.Scatter(
                x=X,
                y=Y,
                name='Scatter',
                mode= 'lines+markers'
                )

        return {'data': [data],'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),
                                                    yaxis=dict(range=[min(Y),max(Y)]),)}

    except Exception as e:
        with open('errors.txt','a') as f:
            f.write(str(e))
            f.write('\n')


@app.callback(Output('discussion','figure'),
             [Input('sentiment_term','value'),
             Input('topic_update', 'n_intervals')]
             )

def getHotWords(sentiment_term,n_intervals):
    
    df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
    df.sort_values('unix', inplace=True)
    
    text = df['tweet'].values
    
    text = str(text)
    
    processed_feature = re.sub(r'\W', ' ', text)

    # remove all single characters
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)

    # Remove single characters from the start
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature) 

    #Substituting multiple spaces with single space
    processed_feature = re.sub(r'\s+', ' ', processed_feature, flags=re.I)

    # Converting to Lowercase
    processed_feature = processed_feature.lower()
    
    stop_words = set(stopwords.words('english'))
    
    tokens = processed_feature.split()
    
    c = Counter(tokens)
    
    dict_ = {}
    
    for i,v in c.items():
        if(i in stop_words or len(i) <= 3):
            continue
            
        else:
            dict_[i] = v
        
    data = dict(sorted(dict_.items(),key=lambda x: x[1],reverse=True))

    fig = go.Figure()
    fig.add_trace(
        go.Bar(x = list(data.values())[0:10],
               y = list(data.keys())[0:10],
               marker=go.bar.Marker(
                    color="rgb(253, 240, 54)",
                    line=dict(color="rgb(0, 0, 0)",
                              width=1.25)
                ),
               orientation = "h",
              )
    )

    return fig

@app.callback(Output('sentiment_pie','figure'),
             [Input('sentiment_term','value'),
             Input('pie_sentiment', 'n_intervals')]
)

def update_pie_chart(sentiment_term,n_intervals):

    df = pd.read_sql("SELECT * FROM sentiment WHERE tweet LIKE ? ORDER BY unix DESC LIMIT 1000", conn ,params=('%' + sentiment_term + '%',))
    df.sort_values('unix',inplace = True)
    df.dropna(inplace=True)
    
    def getTag(x):
        if(x <= 0.005):
            return 'neg'
        
        else:
            return 'pos'
        
    df['tags'] = df['sentiment'].apply(lambda x: getTag(x))
    
    pos = len(df[df['tags'] == 'pos'])
    neg = len(df) - pos
    
    labels = ['Positive','Negative']
    values = [pos,neg]
    colors = ['#007F25', '#800000']
    
    trace = go.Pie(labels=labels, values=values,
                   hoverinfo='label+percent', textinfo='value', 
                   textfont=dict(size=20, color='#FFFFFF'),
                   marker=dict(colors=colors, 
                               line=dict(color='#0C0F0A', width=2)))

    return {"data":[trace],'layout' : go.Layout(
                                                  title='Positive vs Negative sentiment for {}'.format(sentiment_term),
                                                  font={'color': '#0C0F0A'},
                                                  showlegend=True)}

if __name__ == '__main__':
    app.run_server(debug=True,use_reloader=False)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Darshan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
from dash.dependencies import Output, Input